**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Feb  5 15:14:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'AutoCrat Demo Folder'	      Initial.rar
'AutoCrat Demo Folder (1)'    item_dedup.csv
 best.pt		      rating.csv
 Certificate.gslides	      yolov3
"Certify'em Records.gsheet"   Yolov3_Trained_for_PubLayNet.ipynb
'Colab Notebooks'	     'YoloV5 Weights'


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14691, done.
remote: Total 14691 (delta 0), reused 0 (delta 0), pack-reused 14691
Receiving objects: 100% (14691/14691), 13.27 MiB | 8.33 MiB/s, done.
Resolving deltas: 100% (9995/9995), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Panda" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-02-05 15:16:17--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  3.04MB/s    in 48s     

2021-02-05 15:17:06 (3.24 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/panda_00177.txt  
  inflating: data/obj/panda_00178.jpg  
  inflating: data/obj/panda_00178.txt  
  inflating: data/obj/panda_00179.jpg  
  inflating: data/obj/panda_00179.txt  
  inflating: data/obj/panda_00180.jpg  
  inflating: data/obj/panda_00180.txt  
  inflating: data/obj/panda_00181.jpg  
  inflating: data/obj/panda_00181.txt  
  inflating: data/obj/panda_00182.jpg  
  inflating: data/obj/panda_00182.txt  
  inflating: data/obj/panda_00183.jpg  
  inflating: data/obj/panda_00183.txt  
  inflating: data/obj/panda_00184.jpg  
  inflating: data/obj/panda_00184.txt  
  inflating: data/obj/panda_00185.jpg  
  inflating: data/obj/panda_00185.txt  
  inflating: data/obj/panda_00186.jpg  
  inflating: data/obj/panda_00186.txt  
  inflating: data/obj/panda_00187.jpg  
  inflating: data/obj/panda_00187.txt  
  inflating: data/obj/panda_00188.jpg  
  inflating: data/obj/panda_00188.txt  
  inflating: data/obj/panda_00189.jpg  
  i

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.442000 0.280637 0.512000 0.526961

['15', '0.442000', '0.280637', '0.512000', '0.526961']
0 data/obj/panda_00005.txt
0 0.442000 0.280637 0.512000 0.526961
15 0.632743 0.505882 0.734513 0.811765

['15', '0.632743', '0.505882', '0.734513', '0.811765']
1 data/obj/panda_00112.txt
0 0.632743 0.505882 0.734513 0.811765
15 0.316000 0.392216 0.212000 0.203593

['15', '0.316000', '0.392216', '0.212000', '0.203593']
2 data/obj/panda_00107.txt
0 0.316000 0.392216 0.212000 0.203593
15 0.656000 0.522455 0.432000 0.553892

['15', '0.656000', '0.522455', '0.432000', '0.553892']
2 data/obj/panda_00107.txt
0 0.656000 0.522455 0.432000 0.553892
15 0.450000 0.463964 0.544000 0.891892

['15', '0.450000', '0.463964', '0.544000', '0.891892']
3 data/obj/panda_00092.txt
0 0.450000 0.463964 0.544000 0.891892
15 0.418000 0.540000 0.676000 0.434667

['15', '0.418000', '0.540000', '0.676000', '0.434667']
4 data/obj/panda_00078.txt
0 0.418000 0.540000 0.676000 0.434667
15 0.445087 0.229167 0.612717 0.358333



In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/panda_00013.jpg', 'data/obj/panda_00032.jpg', 'data/obj/panda_00002.jpg', 'data/obj/panda_00120.jpg', 'data/obj/panda_00198.jpg', 'data/obj/panda_00063.jpg', 'data/obj/panda_00121.jpg', 'data/obj/panda_00175.jpg', 'data/obj/panda_00189.jpg', 'data/obj/panda_00092.jpg', 'data/obj/panda_00128.jpg', 'data/obj/panda_00180.jpg', 'data/obj/panda_00008.jpg', 'data/obj/panda_00146.jpg', 'data/obj/panda_00033.jpg', 'data/obj/panda_00110.jpg', 'data/obj/panda_00129.jpg', 'data/obj/panda_00061.jpg', 'data/obj/panda_00108.jpg', 'data/obj/panda_00006.jpg', 'data/obj/panda_00068.jpg', 'data/obj/panda_00127.jpg', 'data/obj/panda_00144.jpg', 'data/obj/panda_00179.jpg', 'data/obj/panda_00160.jpg', 'data/obj/panda_00200.jpg', 'data/obj/panda_00070.jpg', 'data/obj/panda_00196.jpg', 'data/obj/panda_00192.jpg', 'data/obj/panda_00190.jpg', 'data/obj/panda_00009.jpg', 'data/obj/panda_00044.jpg', 'data/obj/panda_00071.jpg', 'data/obj/panda_00038.jpg', 'data/obj/panda_00115.jpg', 'data/obj/panda_001

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 114464, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.857892), count: 4, class_loss = 0.034687, iou_loss = 0.157185, total_loss = 0.191872 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 114468, rewritten_bbox = 0.000000 % 
v3 (m

You could see that it has been stopped intentionally, Because we got the result at the 1680th Iteration, If you want to increase the accuracy - Resume the run
Output got using the ***yolov3_training_last.weights*** 